In [1]:
# !wget https://huggingface.co/datasets/mesolitica/semisupervised-abstractive-summarization-ms-news/resolve/main/populate-news.json.semisupervised

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import json
import malaya
from tqdm import tqdm
from unidecode import unidecode
import random
import re

In [4]:
months = {
    'january',
    'jan',
    'januari',
    'february',
    'feb',
    'februari',
    'march',
    'mac',
    'april',
    'apr',
    'may',
    'mei',
    'june',
    'jun',
    'july',
    'julai',
    'august',
    'ogos',
    'aug',
    'september',
    'sep',
    'october',
    'oktober',
    'oct',
    'november',
    'nov',
    'december',
    'disember',
    'dec',
    'utusan',
    'malaysiakini',
    'astroawani',
    'bernama',
    'com',
}

In [5]:
def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

In [6]:
before, after = [], []

count = 0
rejected = []
languages = []
accepted = []
para = []
malaysian_news = {
    'kosmo',
    'hmetro',
    'malaymail',
    'projekmm',
    'bharian',
    'utusan',
    'astroawani',
    'themalaysianinsight',
    'malaysiakini',
    'bernama'
}

def reject(data):
    if data['news'] in malaysian_news:
        return False
    if any([n in data['top-image'] for n in malaysian_news]):
        return False
    if any([n in data['url'] for n in malaysian_news]):
        return False
    if 'com.my' in data['top-image']:
        return False
    if data['language'] == 'malay':
        return False
    if 'Siaran Pers' in data['news']:
        return True
    if '.id' in data['news']:
        return True
    
    return True

texts = []
with open('populate-news.json.semisupervised') as fopen:
    for l in tqdm(fopen):
        data = json.loads(l)
        text = re.sub(r'[ ]+', ' ', data['text']).strip()
        if 'kindly register' in text.lower() or 'disabled in your browser' in text.lower():
            continue
        if len(text.split()) < 30:
            continue

        texts.append(data['title'])
        splitted = malaya.text.function.split_into_sentences(data['text'], 2)
        texts.extend(splitted)
            
        count += 1

81717it [00:54, 1511.60it/s]


In [7]:
len(texts)

1187030

In [8]:
from glob import glob
from bs4 import BeautifulSoup

files = sorted(glob('*.nested.semisupervised'))
files

['berita-bisnes.json.nested.semisupervised',
 'berita-dunia.json.nested.semisupervised',
 'berita-hiburan.json.nested.semisupervised',
 'berita-malaysia.json.nested.semisupervised',
 'berita-politik.json.nested.semisupervised',
 'berita-sukan.json.nested.semisupervised',
 'berita-teknologi.json.nested.semisupervised',
 'gaya-hidup.json.nested.semisupervised']

In [9]:
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            soup = BeautifulSoup(data['r']['response']['articleBody'], "lxml")
            text = BeautifulSoup(soup.text, 'lxml').text
            text = re.sub(r'[ ]+', ' ', text).strip()
            if 'kindly register' in text.lower() or 'disabled in your browser' in text.lower():
                continue
            if len(text.split()) < 30:
                continue
                
            texts.append(data['original']['title'])
            texts.append(data['original']['description'])
            splitted = malaya.text.function.split_into_sentences(text, 2)
            texts.extend(splitted)

9622it [00:12, 786.74it/s]
7897it [00:08, 905.72it/s] 
9956it [00:14, 702.30it/s]
8578it [00:10, 788.17it/s]
9970it [00:13, 743.46it/s]
9826it [00:11, 822.69it/s]
972it [00:01, 756.50it/s]
5342it [00:08, 603.67it/s]


In [10]:
len(texts)

2894239

In [12]:
texts_ = [t for t in texts if len(t) > 3]

In [14]:
len(texts_)

2187573

In [17]:
with open('malay-news.txt', 'w') as fopen:
    fopen.write('\n'.join(texts_))

In [18]:
!split -l 100000 -d --additional-suffix=.splitted malay-news.txt malay-news.txt